# <font color='red'>Objetos</font>

In [1]:
# %run gear_db.ipynb
from decorador_db_gen import db_generator

In [2]:
dbg = db_generator()

Database carregado com sucesso!


In [5]:
equip_database = dbg.getEquip_DB()
hat_database = dbg.getHat_DB()
shield_database = dbg.getShield_DB()
robe_database = dbg.getRobe_DB()
armor_database = dbg.getArmor_DB()
shoes_database = dbg.getShoes_DB()
acessory_database = dbg.getAcessory_DB()

In [ ]:
class Equip:
    def __init__(self, unique_id, name, main_type, subtype, weight, base_attack, slots, joblist, weapon_level, min_level, refineable):
        print("olá")

In [ ]:
class PlayerBuild:
    def __init__(self, base_level, job_level, current_job, statbuild):
        self.base_level = base_level
        self.job_level = job_level
        self.current_job = current_job
        self.statbuild = statbuild
        self.str = statbuild[0];	self.str_bonus = 0;	self.core_str = self.str
        self.agi = statbuild[1];	self.agi_bonus = 0;	self.core_agi = self.agi
        self.vit = statbuild[2];	self.vit_bonus = 0;	self.core_vit = self.vit
        self.int = statbuild[3];	self.int_bonus = 0;	self.core_int = self.int
        self.dex = statbuild[4];	self.dex_bonus = 0;	self.core_dex = self.dex
        self.luk = statbuild[5];	self.luk_bonus = 0;	self.core_luk = self.luk
        
        #arquivos
        self.hp_df = pd.read_csv ('max_hp_table.csv')
        self.job_bonuses = job_bonuses_list[current_job]['FULL_BONUSES']
        self.max_job = job_bonuses_list[current_job]['MAX_JOB']
        
        # status com o ajuste do job
        self.str_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['STR'])
        self.agi_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['AGI'])
        self.vit_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['VIT'])
        self.int_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['INT'])
        self.dex_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['DEX'])
        self.luk_bonus = self.evaluate_stat_bonus(self.job_level, job_bonuses_list[current_job]['LUK'])
        
        # somatórios de status
        self.str += self.str_bonus; self.agi += self.agi_bonus; self.vit += self.vit_bonus;
        self.int += self.int_bonus; self.dex += self.dex_bonus; self.luk += self.luk_bonus;
        
        # esquiva
        self.flee_bonuses = 0
        self.flee = self.agi + self.flee_bonuses + self.base_level
        self.perfect_flee = 1
        
        # precisão
        self.hit_bonuses = 0
        self.hit = self.dex + self.hit_bonuses + self.base_level
        
        # crítico
        self.crit_bonuses = 0
        self.crit_rate = 1 + math.floor(1/3*self.luk + self.crit_bonuses)
        self.crit_target_luk = 0
        self.crit_finalrate = self.crit_rate - (self.crit_target_luk*0.2)
        
        # cálculo do HP
        self.trans_mod = 1
        self.aditive_modifiers = 0
        self.multiplicative_modifiers = 0
        self.base_hp = list(self.hp_df[self.current_job])[self.base_level]
        self.max_hp = self.calculate_max_hp()
        
        # cálculo do SP
        self.sp_mod_a = 0
        self.sp_mod_b = 0
        self.max_sp = self.calculate_max_sp()
        
    def calculate_base_hp(self):
        hp_base_table = {"novice": (0, 5), "super_novice": (0, 5), "swordsman": (0.7, 5), "archer": (0.5, 5), "thief": (0.5, 5), "acolyte": (0.4, 5), "merchant": (0.4, 5), "mage": (0.3, 5), "knight": (1.5, 5), "hunter": (0.85, 5), "assassin": (1.1, 5), "priest": (0.75, 5), "blacksmith": (0.9, 5), "wizard": (0.55, 5), "crusader": (1.1, 7), "dancer": (0.75, 3), "bard": (0.75, 3), "rogue": (0.85, 5), "monk": (0.9, 6.5), "alchemist": (0.9, 5), "sage": (0.75, 5)}
        hp_job_a = hp_base_table[self.current_job][0]
        hp_job_b = hp_base_table[self.current_job][1]
        base_hp = 35 + (self.base_level * hp_job_b)
        for i in range(2, self.base_level + 1):
            base_hp = base_hp + round(hp_job_a * i)
        return (base_hp)
    
    def calculate_max_hp(self):
        max_hp = math.floor(self.base_hp * (1+(0.01*self.vit)))
        max_hp += self.aditive_modifiers
        max_hp = math.floor(max_hp * (1 + (self.multiplicative_modifiers*0.01)))
        return max_hp
    
    def calculate_max_sp(self):
        sp_base_table = {"novice": (1), "super_novice": (1), "swordsman": (2), "archer": (2), "thief": (2), "acolyte": (5), "merchant": (3), "mage": (6), "knight": (3), "hunter": (4), "assassin": (4), "priest": (8), "blacksmith": (4), "wizard": (9), "crusader": (4.7), "dancer": (6), "bard": (6), "rogue": (5), "monk": (4.7), "alchemist": (4), "sage": (7)}
        base_sp = 10
        base_sp += math.floor(self.base_level * sp_base_table[self.current_job])
        max_sp = base_sp
        max_sp = math.floor(max_sp * (1 + (self.int*0.01)))
        max_sp += self.sp_mod_a
        max_sp = math.floor(max_sp * (1 + (self.sp_mod_b * 0.01)))
        return max_sp
    
    def evaluate_stat_bonus(self, job_level, stat_array):
        if(not stat_array):
            return 0
        for i in range(len(stat_array) -1, -1, -1):
            k = i + 1
            if(job_level >= stat_array[i]):
                break
        return k
    
    def printbuild(self):
        if(self.str_bonus == 0):
            print("	[str] → {}".format(self.core_str))
        else:
            print('	[str] → {} +{}'.format(self.core_str, self.str_bonus))
            
        if(self.agi_bonus == 0):
            print("	[agi] → {}".format(self.core_agi))
        else:
            print('	[agi] → {} +{}'.format(self.core_agi, self.agi_bonus))
            
        if(self.vit_bonus == 0):
            print("	[vit] → {}".format(self.core_vit))
        else:
            print('	[vit] → {} +{}'.format(self.core_vit, self.vit_bonus))
            
        if(self.int_bonus == 0):
            print("	[int] → {}".format(self.core_int))
        else:
            print('	[int] → {} +{}'.format(self.core_int, self.int_bonus))
            
        if(self.dex_bonus == 0):
            print("	[dex] → {}".format(self.core_dex))
        else:
            print('	[dex] → {} +{}'.format(self.core_dex, self.dex_bonus))
            
        if(self.luk_bonus == 0):
            print("	[luk] → {}".format(self.core_luk))
        else:
            print('	[luk] → {} +{}'.format(self.core_luk, self.luk_bonus))
        

In [ ]:
p1 = PlayerBuild(99, 50, 'assassin', [1, 1, 1, 1, 1, 30])
print("max_hp → {}".format(p1.max_hp))
print("max_sp → {}".format(p1.max_sp))
p1.printbuild()
print("hit → {}".format(p1.hit))
print("flee → {} +{}".format(p1.flee, p1.perfect_flee))
print("critical → {}".format(p1.crit_rate))

# <font color='red'>Criar sistema de escudos</font>

In [ ]:
equip_database[2102]

In [ ]:
equip_database[2114]

In [ ]:
class Shield:
    def __init__(self, shield_id, name = '', weight = 0, defense = 0, slots = 0, jobs = {'All': True}, classes = {'All': True}, gender = 'Both', locations = 'None', equipLevelMin = 0, refineable = False, view = 0, script = ""):
        self.shield_id = shield_id
        self.name = name
        self.weight = weight
        self.defense = defense
        self.slots = slots
        self.jobs = jobs
        self.classes = classes
        self.gender = gender
        self.locations = locations
        self.equipLevelMin = equipLevelMin
        self.refineable = refineable
        self.view = view
        self.script = script
        
        default_param = ['', 0, 0, 0, {'All': True}, {'All': True}, 'Both', 'None', 0, False, 0, '']
        
    def show_param(self):
        print('shield_id → {}'.format(self.shield_id)); print('name → {}'.format(self.name)); print('weight → {}'.format(self.weight)); print('defense → {}'.format(self.defense)); print('slots → {}'.format(self.slots)); print('jobs → {}'.format(self.jobs)); print('classes → {}'.format(self.classes)); print('gender → {}'.format(self.gender)); print('locations → {}'.format(self.locations)); print('equipLevelMin → {}'.format(self.equipLevelMin)); print('refineable → {}'.format(self.refineable)); print('view → {}'.format(self.view)); print('script → {}'.format(self.script)); 
        
        
s1 = Shield(234)

In [ ]:
def normalize_missing_params(input_db):
    copy_db = input_db
    checklist = {'Id': 0, 'Name': '', 'Weight': 0, 'Defense': 0, 'Slots': 0, 'Jobs': {'All': True}, 'Classes': {'All': True}, 'Gender': 'Both', 'Locations': 'None', 'EquipLevelMin': 0, 'Refineable': False, 'View': 0, 'Script': 0}
    for i in checklist.keys():
        if(i not in input_db):
            copy_db[i] = checklist[i]
    return copy_db

shield_database = {}
for i in equip_database.values():
    if('Locations' in i and 'Type' in i):
        if(i['Locations'] == {'Left_Hand': True} and i['Type'] == 'Armor'):
            print('id = {}'.format(i['Id']))
            print('conteúdo → {}\n'.format(i))
            shield_database[i['Id']] = normalize_missing_params(i)

In [ ]:
shield_database[2131]

In [ ]:
class PlayerEquips:
    def __init__(self, tophat, midhat, lowhat, armor, weapon, shield, robe, shoes, acessory1, acessory2):
        self.tophat = tophat
        self.midhat = midhat
        self.lowhat = lowhat
        self.armor = armor
        self.weapon = weapon
        self.shield = shield
        self.robe = robe
        self.shoes = shoes
        self.acessory1 = acessory1
        self.acessory2 = acessory2